#### 1. 패키지 설치

In [ ]:
# %pip install -q -U langchain langchain-ollama langgraph

#### 로컬 Ollama로 설치한 deepseek-r1 모델과 ExaOne3 또는 Qwen2.5 모델을 사용하기
##### ollama run deepseek-r1:7b
##### ollama run exaone3.5
##### ollama run qwen2.5:1.5b

##### 최신버전 LangChain에서는 ChatOllama와 RunnableSequence(prompt | llm) 를 사용

##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (영어로 질문, invoke()함수)

In [69]:
import requests

response = requests.get("http://127.0.0.1:11434")
print(response.text)

Ollama is running


In [68]:
from langchain_ollama import ChatOllama

try:
    deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)    # 모델 호출
    response = deepseek.invoke("which is bigger between 9.9 and 9.11?")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")

<think>
To determine which number is larger between 9.9 and 9.11, I first recognize that both numbers have the same integer part, which is 9.

Next, I compare their decimal parts by aligning them to the thousandth place. The number 9.9 can be written as 9.900, so its decimal part becomes 0.900.

Similarly, the number 9.11 can be expressed as 9.110, giving it a decimal part of 0.110.

Comparing these two decimals, 0.900 is greater than 0.110.
</think>

To determine which number is larger between **9.9** and **9.11**, let's compare them step by step.

### Step 1: Align the Decimal Places
Both numbers have the same integer part (9). To make comparison easier, we can write them with the same number of decimal places:

\[
9.9 \quad \text{and} \quad 9.11
\]

This can be rewritten as:

\[
9.90 \quad \text{and} \quad 9.11
\]

### Step 2: Compare the Decimal Parts
Now, compare the decimal parts of both numbers:

- **9.90** has a decimal part of **0.90**
- **9.11** has a decimal part of **0.11**

##### exaone3.4 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, invoke()함수)

In [70]:
from langchain_ollama import ChatOllama

try:
    exaone = ChatOllama(model="exaone3.5:2.4b", temperature=0.5, n_gpu_layers=0, batch_size=128)
    #exaone = ChatOllama(model="exaone3.5:2.4b", temperature=0.5)
    # 모델 호출
    response = exaone.invoke("9.9와 9.11 중 무엇이 더 큰가요?")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")

Error: model "exaone3.5:2.4b" not found, try pulling it first (status code: 404)


##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (영어로 질문, stream()함수)

In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama

deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)

answer = []
for chunk in deepseek.stream("which is bigger between 9.9 and 9.11?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)


In [ ]:

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))

##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, stream()함수)

In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama

deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)

answer = []
for chunk in deepseek.stream("9.9와 9.11 중 무엇이 더 큰가요?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))

##### Exaone 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, stream()함수)

In [71]:
from langchain_ollama import ChatOllama

#model = ChatOllama(model="exaone3.5:2.4b", temperature=0.5)
model = ChatOllama(model="qwen2.5:1.5b", temperature=0.5)

answer = []
for chunk in model.stream("9.9와 9.11 중 무엇이 더 큰가요?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))


9.9보다 9.11 작입니다.

9.9보다 9.11 작입니다.

#### DeepSeek의 추론 능력과 ExaOne의 한글 생성 능력 결합하기
* DeepSeek는 태그 안에서 이루어지는 추론을 기반으로 다른 LLM 대비 높은 성능을 발휘합니다.
* 하지만 Ollama에서 제공하는 deepseek r1-distill-qwen 모델은 한국어 생성 능력이 부족합니다.
* DeepSeek의 추론 능력과 ExaOne의 한글 생성 능력 결합해 보겠습니다.

In [72]:
from langchain_ollama import ChatOllama

reasoning_model = ChatOllama(model="deepseek-r1:1.5b", temperature=0, stop=["</think>"])
print(reasoning_model)

model='deepseek-r1:1.5b' temperature=0.0 stop=['</think>']


In [73]:
#generation_model = ChatOllama(model="exaone3.5:2.4b", temperature=0.7)
generation_model = ChatOllama(model="qwen2.5:1.5b", temperature=0.7)
print(generation_model)

model='qwen2.5:1.5b' temperature=0.7


#### LangGraph 로 연결하기

In [76]:
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate

#LangGraph에서 State 사용자정의 클래스는 노드 간의 정보를 전달하는 틀입니다. 
#노드 간에 계속 전달하고 싶거나, 그래프 내에서 유지해야 할 정보를 미리 정의힙니다. 
class State(TypedDict):
    question: str
    thinking: str
    answer: str

answer_prompt = ChatPromptTemplate([
    (
        "system",
        """
        당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.

        당신의 작업은 다음과 같습니다:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.

        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.

        목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.
        """
    ),
    (
        "human",
        """
        질문: {question}
        추론: {thinking}
        """
    )
])
print(answer_prompt)

input_variables=['question', 'thinking'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n        당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.\n\n        당신의 작업은 다음과 같습니다:\n        - 질문과 제공된 추론을 신중하게 분석하세요.\n        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.\n        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.\n        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.\n\n        지침:\n        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.\n        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.\n        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.\n        - 도움이 되고 전문적인 톤을 유지하세요.\n\n        목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'thinking'], input_types={}, partial_variables={}, template='\n        질문: {question}\n        추론: {thinking}\n        '), additional_kwargs={})]


In [77]:
#DeepSeek를 통해서 추론 부분까지만 생성합니다. 
def think(state: State):
    question = state["question"]
    response = reasoning_model.invoke(question)
    #print(response.content)
    return {"thinking": response.content}

#Qwen를 통해서 결과 출력 부분을 생성합니다.
def generate(state: State):
    messages = answer_prompt.invoke({"question": state["question"], "thinking": state["thinking"]})
    response = generation_model.invoke(messages)
    print(response.content)
    return {"answer": response.content}

# 그래프 컴파일
graph_builder = StateGraph(State).add_sequence([think, generate])
graph_builder.add_edge(START, "think")
graph = graph_builder.compile()

# 입력 데이터
inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}

# invoke()를 사용하여 그래프 호출
result = graph.invoke(inputs)
print(result)

# 결과 출력
print("==> 생성된 답변: \n")
print(result["answer"])

9.9은 9.11보다 더 큰 수입니다. 이는 9.9의 9와 9.11의 9를 비교했을 때, 9.9의 9가 9.11의 9보다 크다는 이유에서 나타납니다. 즉, 9.9의 90분이 9.11의 11분보다 커서, 9.9는 9.11보다 큰 수입니다.
{'question': '9.9와 9.11 중 무엇이 더 큰가요?', 'thinking': "<think>\nFirst, I need to compare the two numbers: 9.9 and 9.11.\n\nTo make an accurate comparison, it's helpful to express both numbers with the same number of decimal places. I can write 9.9 as 9.90.\n\nNow, comparing 9.90 and 9.11:\n\n- The whole number part is the same (both have 9).\n- Comparing the tenths place: 9 in 9.90 versus 1 in 9.11.\n- Since 9 is greater than 1, 9.90 is larger than 9.11.\n\nTherefore, 9.9 is greater than 9.11.\n", 'answer': '9.9은 9.11보다 더 큰 수입니다. 이는 9.9의 9와 9.11의 9를 비교했을 때, 9.9의 9가 9.11의 9보다 크다는 이유에서 나타납니다. 즉, 9.9의 90분이 9.11의 11분보다 커서, 9.9는 9.11보다 큰 수입니다.'}
==> 생성된 답변: 

9.9은 9.11보다 더 큰 수입니다. 이는 9.9의 9와 9.11의 9를 비교했을 때, 9.9의 9가 9.11의 9보다 크다는 이유에서 나타납니다. 즉, 9.9의 90분이 9.11의 11분보다 커서, 9.9는 9.11보다 큰 수입니다.


In [78]:
import os
import sys
from langchain_ollama import ChatOllama
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate

# UTF-8 인코딩 강제 설정 (Jupyter 노트북 호환)
os.environ['PYTHONIOENCODING'] = 'utf-8'
os.environ['LANG'] = 'ko_KR.UTF-8'
os.environ['LC_ALL'] = 'ko_KR.UTF-8'

# Jupyter 환경에서는 reconfigure 대신 환경변수로 처리
try:
    if hasattr(sys.stdout, 'reconfigure') and sys.stdout.encoding != 'utf-8':
        sys.stdout.reconfigure(encoding='utf-8')
except (AttributeError, OSError):
    # Jupyter 노트북이나 다른 환경에서는 패스
    pass

# 모델 초기화 - 한글 처리 개선
reasoning_model = ChatOllama(
    model="deepseek-r1:1.5b", 
    temperature=0, 
    stop=["</think>"]
)

generation_model = ChatOllama(
    model="qwen2.5:1.5b", 
    temperature=0.7
)

# LangGraph State 정의
class State(TypedDict):
    question: str
    thinking: str
    answer: str

# 개선된 프롬프트 템플릿
answer_prompt = ChatPromptTemplate([
    (
        "system",
        """당신은 한국어로 응답하는 AI 어시스턴트입니다. 
        반드시 한국어로만 답변하세요.
        
        당신의 작업:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 한국어 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.
        
        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.
        
        중요: 반드시 한국어로만 응답하세요."""
    ),
    (
        "human",
        """질문: {question}
        
        추론 과정: {thinking}
        
        위 내용을 바탕으로 한국어로 답변해주세요:"""
    )
])

def ensure_utf8_string(text):
    """문자열이 UTF-8로 제대로 인코딩되었는지 확인하고 변환"""
    if text is None:
        return ""
    if isinstance(text, bytes):
        return text.decode('utf-8', errors='ignore')
    return str(text)

# DeepSeek를 통해서 추론 부분까지만 생성
def think(state: State):
    question = state["question"]
    print(f"[DEBUG] 입력 질문: {question}")
    print(f"[DEBUG] 질문 타입: {type(question)}")
    
    response = reasoning_model.invoke(question)
    thinking_content = ensure_utf8_string(response.content)
    
    print(f"[DEBUG] 추론 결과 타입: {type(response.content)}")
    print(f"[DEBUG] 추론 결과 길이: {len(thinking_content)}")
    print(f"[DEBUG] 추론 결과 미리보기: {thinking_content[:200]}...")
    
    return {"thinking": thinking_content}

# qwen2.5를 통해서 결과 출력 부분을 생성
def generate(state: State):
    question = ensure_utf8_string(state["question"])
    thinking = ensure_utf8_string(state["thinking"])
    
    print(f"[DEBUG] generate 함수 - 질문: {question}")
    print(f"[DEBUG] generate 함수 - 추론 길이: {len(thinking)}")
    print(f"[DEBUG] generate 함수 - 추론 미리보기: {thinking[:200]}...")
    
    messages = answer_prompt.invoke({
        "question": question, 
        "thinking": thinking
    })
    
    print(f"[DEBUG] 프롬프트 메시지 생성 완료")
    
    response = generation_model.invoke(messages)
    answer_content = ensure_utf8_string(response.content)
    
    print(f"[DEBUG] 최종 응답 타입: {type(response.content)}")
    print(f"[DEBUG] 최종 응답 길이: {len(answer_content)}")
    print(f"[DEBUG] 최종 응답 내용: {answer_content}")
    
    return {"answer": answer_content}

# 그래프 구성 및 컴파일
graph_builder = StateGraph(State).add_sequence([think, generate])
graph_builder.add_edge(START, "think")
graph = graph_builder.compile()

def main():
    # 입력 데이터
    inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}
    
    print("="*50)
    print("LangGraph 실행 시작")
    print("="*50)
    
    try:
        # invoke()를 사용하여 그래프 호출
        result = graph.invoke(inputs)
        
        print("="*50)
        print("실행 결과")
        print("="*50)
        print(f"전체 결과: {result}")
        print(f"최종 답변: {result.get('answer', '답변 없음')}")
        
    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

LangGraph 실행 시작
[DEBUG] 입력 질문: 9.9와 9.11 중 무엇이 더 큰가요?
[DEBUG] 질문 타입: <class 'str'>
[DEBUG] 추론 결과 타입: <class 'str'>
[DEBUG] 추론 결과 길이: 431
[DEBUG] 추론 결과 미리보기: <think>
First, I need to compare the two numbers: 9.9 and 9.11.

Both numbers have the same whole number part, which is 9.

To make a fair comparison, I'll align their decimal places by writing 9.9 as...
[DEBUG] generate 함수 - 질문: 9.9와 9.11 중 무엇이 더 큰가요?
[DEBUG] generate 함수 - 추론 길이: 431
[DEBUG] generate 함수 - 추론 미리보기: <think>
First, I need to compare the two numbers: 9.9 and 9.11.

Both numbers have the same whole number part, which is 9.

To make a fair comparison, I'll align their decimal places by writing 9.9 as...
[DEBUG] 프롬프트 메시지 생성 완료
[DEBUG] 최종 응답 타입: <class 'str'>
[DEBUG] 최종 응답 길이: 214
[DEBUG] 최종 응답 내용: 먼저, 두 숫자 9.9와 9.11를 비교해야 합니다.

이俩의 전체 소수부는 모두 9입니다.

정확한 비교를 위해서, 9.9를 9.90으로 표시하겠습니다.

그러면 각 자리에서 비교해 보겠습니다:

- 페트릭스: 이俩 모두 9입니다.
- 셋째 자리: 9.9는 9이고 9.11의 1이 더 큽니다. 

따라서, 9.90은 9.11보다 크습니다.

그래서 9.9는 9.11 보다 큰답니다.
실행 결과
전체 결과:

In [85]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
import pyppeteer

pyppeteer.launcher.CHROMIUM_DOWNLOAD = False
executable_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"

display(
  Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.PYPPETEER
        )
    )
)

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [ ]:
inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}

async for event in graph.astream_events(inputs, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event['data']['chunk'].content, end="", flush=True)

In [88]:
import gradio as gr
import os
import sys
from langchain_ollama import ChatOllama
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate

# UTF-8 인코딩 강제 설정 (Jupyter 노트북 호환)
os.environ['PYTHONIOENCODING'] = 'utf-8'
os.environ['LANG'] = 'ko_KR.UTF-8'
os.environ['LC_ALL'] = 'ko_KR.UTF-8'

# Jupyter 환경에서는 reconfigure 대신 환경변수로 처리
try:
    if hasattr(sys.stdout, 'reconfigure') and sys.stdout.encoding != 'utf-8':
        sys.stdout.reconfigure(encoding='utf-8')
except (AttributeError, OSError):
    # Jupyter 노트북이나 다른 환경에서는 패스
    pass

# 모델 설정: 두 개의 서로 다른 모델을 사용하여 추론과 답변 생성을 수행
# - reasoning_model: 추론을 담당하는 모델 (온도 낮음, 정확한 분석용)
# - generation_model: 답변 생성을 담당하는 모델 (온도 높음, 창의적 응답용)
reasoning_model = ChatOllama(
    model="deepseek-r1:1.5b", 
    temperature=0, 
    stop=["</think>"]
)

generation_model = ChatOllama(
    model="qwen2.5:1.5b", 
    temperature=0.7
)

# 상태(State) 정의: 그래프에서 상태를 유지하기 위한 데이터 구조
class State(TypedDict):
    question: str   # 사용자의 질문
    thinking: str   # 추론 결과
    answer: str     # 최종 답변

# 개선된 프롬프트 템플릿
answer_prompt = ChatPromptTemplate([
    (
        "system",
        """당신은 한국어로 응답하는 AI 어시스턴트입니다. 
        반드시 한국어로만 답변하세요.
        
        당신의 작업:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 한국어 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.
        
        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.
        
        중요: 반드시 한국어로만 응답하세요."""
    ),
    (
        "human",
        """질문: {question}
        
        추론 과정: {thinking}
        
        위 내용을 바탕으로 한국어로 답변해주세요:"""
    )
])


def ensure_utf8_string(text):
    """문자열이 UTF-8로 제대로 인코딩되었는지 확인하고 변환"""
    if text is None:
        return ""
    if isinstance(text, bytes):
        try:
            return text.decode('utf-8')
        except UnicodeDecodeError:
            return text.decode('utf-8', errors='ignore')
    
    # 문자열이지만 인코딩 문제가 있을 수 있는 경우 처리
    if isinstance(text, str):
        try:
            # 문자열을 UTF-8로 인코딩했다가 다시 디코딩하여 정리
            return text.encode('utf-8').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            return text
    
    return str(text)

# DeepSeek를 통해서 추론 부분까지만 생성
def think(state: State):
    question = state["question"]
    print(f"[DEBUG] 입력 질문: {question}")
    print(f"[DEBUG] 질문 타입: {type(question)}")
    
    response = reasoning_model.invoke(question)
    thinking_content = ensure_utf8_string(response.content)
    
    print(f"[DEBUG] 추론 결과 타입: {type(response.content)}")
    print(f"[DEBUG] 추론 결과 길이: {len(thinking_content)}")
    print(f"[DEBUG] 추론 결과 미리보기: {thinking_content[:200]}...")
    
    return {"thinking": thinking_content}

# qwen2.5를 통해서 결과 출력 부분을 생성
def generate(state: State):
    question = ensure_utf8_string(state["question"])
    thinking = ensure_utf8_string(state["thinking"])
    
    print(f"[DEBUG] generate 함수 - 질문: {question}")
    print(f"[DEBUG] generate 함수 - 추론 길이: {len(thinking)}")
    print(f"[DEBUG] generate 함수 - 추론 미리보기: {thinking[:200]}...")
    
    messages = answer_prompt.invoke({
        "question": question, 
        "thinking": thinking
    })
    
    print(f"[DEBUG] 프롬프트 메시지 생성 완료")
    
    response = generation_model.invoke(messages)
    answer_content = ensure_utf8_string(response.content)
    
    print(f"[DEBUG] 최종 응답 타입: {type(response.content)}")
    print(f"[DEBUG] 최종 응답 길이: {len(answer_content)}")
    print(f"[DEBUG] 최종 응답 내용: {answer_content}")
    
    return {"answer": answer_content}

# 그래프 생성 함수: 상태(State) 간의 흐름을 정의
def create_graph():
    graph_builder = StateGraph(State).add_sequence([think, generate])
    graph_builder.add_edge(START, "think")
    return graph_builder.compile()

# Gradio 인터페이스 생성 및 실행
def chatbot_interface(message, history):
    graph = create_graph()
    inputs = {"question": message}
    result = graph.invoke(inputs)
    return result["answer"]

# iface = gr.ChatInterface(fn=chatbot_interface, title="AI 챗봇", description="질문을 입력하면 AI가 답변을 제공합니다.")

# Gradio 인터페이스 설정
def launch_gradio():
    # iface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="AI 챗봇", description="질문을 입력하면 AI가 답변을 제공합니다.")
    iface = gr.ChatInterface(fn=chatbot_interface, title="AI 챗봇", description="질문을 입력하면 AI가 답변을 제공합니다.")
    iface.launch()

if __name__ == "__main__":
    #iface.launch()
    launch_gradio()

c:\Users\winds\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.13\Lib\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


[DEBUG] 입력 질문: 9.11과 9.9를 비교하시오
[DEBUG] 질문 타입: <class 'str'>
[DEBUG] 추론 결과 타입: <class 'str'>
[DEBUG] 추론 결과 길이: 342
[DEBUG] 추론 결과 미리보기: <think>
First, I need to compare the two numbers: 9.11 and 9.9.

I'll start by looking at the whole number part of both numbers. Both have a 9 in the units place, so they are equal up to this point.

...
[DEBUG] generate 함수 - 질문: 9.11과 9.9를 비교하시오
[DEBUG] generate 함수 - 추론 길이: 342
[DEBUG] generate 함수 - 추론 미리보기: <think>
First, I need to compare the two numbers: 9.11 and 9.9.

I'll start by looking at the whole number part of both numbers. Both have a 9 in the units place, so they are equal up to this point.

...
[DEBUG] 프롬프트 메시지 생성 완료
[DEBUG] 최종 응답 타입: <class 'str'>
[DEBUG] 최종 응답 길이: 294
[DEBUG] 최종 응답 내용: 네, 9.11과 9.9를 비교해 보겠습니다.

먼저 두 수의 전부가 같은 부분을 살펴볼 필요가 있습니다. 두 숫자 모두 소수점 아래의 9를 가집니다. 따라서 이 단계에서는 두 수가 같아서 이전에 같았다고 볼 수 있습니다.

다음으로, 각 수의 소数점이 나타내는 것들을 확인하겠습니다. 9.11에서 tenths(十分位) 자리에는 1이 있고, 9.9에서 tenths(十分位) 자리에는 9가 있습니다. 이 중 1은 9보다 작으므로, 9.11의 소수점 부분이 더 